# Introducción
Al operador de telecomunicaciones Interconnect le gustaría poder pronosticar su tasa de cancelación de clientes. Si se descubre que un usuario o usuaria planea irse, se le ofrecerán códigos promocionales y opciones de planes especiales. El equipo de marketing de Interconnect ha recopilado algunos de los datos personales de sus clientes, incluyendo información sobre sus planes y contratos.

* __Servicios de Interconnect__

Interconnect proporciona principalmente dos tipos de servicios:

1. Comunicación por teléfono fijo. El teléfono se puede conectar a varias líneas de manera simultánea.
2. Internet. La red se puede configurar a través de una línea telefónica (DSL, *línea de abonado digital*) o a través de un cable de fibra óptica.

Algunos otros servicios que ofrece la empresa incluyen:

- Seguridad en Internet: software antivirus (*ProtecciónDeDispositivo*) y un bloqueador de sitios web maliciosos (*SeguridadEnLínea*).
- Una línea de soporte técnico (*SoporteTécnico*).
- Almacenamiento de archivos en la nube y backup de datos (*BackupOnline*).
- Streaming de TV (*StreamingTV*) y directorio de películas (*StreamingPelículas*)

La clientela puede elegir entre un pago mensual o firmar un contrato de 1 o 2 años. Puede utilizar varios métodos de pago y recibir una factura electrónica después de una transacción.

* __Descripción de los datos__

Los datos consisten en archivos obtenidos de diferentes fuentes:

- `contract.csv` — información del contrato;
- `personal.csv` — datos personales del cliente;
- `internet.csv` — información sobre los servicios de Internet;
- `phone.csv` — información sobre los servicios telefónicos.

En cada archivo, la columna `customerID` (ID de cliente) contiene un código único asignado a cada cliente. La información del contrato es válida a partir del 1 de febrero de 2020.

## Importación

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Carga de datos

In [2]:
path = r'C:\Users\LENOVO\Desktop\python\data_science\Tripleten\Proyecto_final\Data_raw\final_provider'

In [3]:
contract_df = pd.read_csv(path+'\contract.csv')
internet_df = pd.read_csv(path+'\internet.csv')
personal_df = pd.read_csv(path+'\personal.csv')
phone_df = pd.read_csv(path+'\phone.csv')

## EDA

### DataFrame contract_df

In [4]:
contract_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB


In [5]:
display(contract_df.head())

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65


* El DataFrame __contract_df__ constaba de 7043 filas y 8 columnas y no mostró datos ausentes. 
* Las columnas __BeginDate__ deberían contener variables del tipo __datetime__ y la columna __TotalCharges__ del tipo __float__ y ambas contienen tipo __object__. 
* La columna __EndDate__ contiene variables tipo __object__ sin embargo al ser la columna objetivo en el actual problema de clasificación la respuesta __'No'__ por __1__ y el resto de respuestas a __0__ puesto que se trata de clientes que cancelaron el contrato. 
* La columna __PaymentMethod__ un posible valor es __Bank transfer (automatic)__ la cual podría ser redundante si es el único tipo de de transferencia bancaria que existe y podría simplificarse a __Bank transfer__ en tal caso. 
* Las columnas __Type__, __PaperlessBilling__ y __PaymenteMethod__ contiene valores que no respeta el formato snakecase por lo que se modificaron para respetar dicho formato y entonces realizar un análisis para evidenciar la existencia de duplicados.

### Corregir datos

#### Columna 'BeginDate'
Se cambió el tipo de variable de __object__ a tipo __datetime__

In [6]:
print(contract_df['BeginDate'].unique())

['2020-01-01' '2017-04-01' '2019-10-01' '2016-05-01' '2019-09-01'
 '2019-03-01' '2018-04-01' '2019-04-01' '2017-07-01' '2014-12-01'
 '2019-01-01' '2018-10-01' '2015-04-01' '2015-09-01' '2018-01-01'
 '2014-05-01' '2015-10-01' '2014-03-01' '2018-05-01' '2019-11-01'
 '2019-02-01' '2016-01-01' '2017-08-01' '2015-12-01' '2014-02-01'
 '2018-06-01' '2019-12-01' '2017-11-01' '2019-06-01' '2016-04-01'
 '2017-02-01' '2018-12-01' '2014-04-01' '2018-09-01' '2014-11-01'
 '2016-07-01' '2015-02-01' '2018-07-01' '2014-08-01' '2016-03-01'
 '2018-08-01' '2014-10-01' '2015-06-01' '2016-08-01' '2019-05-01'
 '2017-03-01' '2016-02-01' '2017-09-01' '2014-09-01' '2017-12-01'
 '2016-12-01' '2017-06-01' '2015-05-01' '2016-10-01' '2016-09-01'
 '2019-08-01' '2019-07-01' '2017-05-01' '2017-10-01' '2014-07-01'
 '2018-03-01' '2015-01-01' '2018-11-01' '2015-03-01' '2018-02-01'
 '2016-06-01' '2015-08-01' '2015-11-01' '2014-06-01' '2017-01-01'
 '2015-07-01' '2020-02-01' '2016-11-01' '2013-11-01' '2014-01-01'
 '2013-10-

In [7]:
contract_df['BeginDate'] = pd.to_datetime(contract_df['BeginDate'], format='%Y-%m-%d')
contract_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customerID        7043 non-null   object        
 1   BeginDate         7043 non-null   datetime64[ns]
 2   EndDate           7043 non-null   object        
 3   Type              7043 non-null   object        
 4   PaperlessBilling  7043 non-null   object        
 5   PaymentMethod     7043 non-null   object        
 6   MonthlyCharges    7043 non-null   float64       
 7   TotalCharges      7043 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 440.3+ KB


#### Columna 'EndDate'
Se filtró la columna para que sólo se seleccionaran las filas que no contengan el valor __No__ y se reemplaza por __1__ mientras que el resto se reemplaza por __0__. Esto debido a que se abordó un problema de clasificación en el cual la columna target sería esta.

In [8]:
print(contract_df['EndDate'].value_counts())

EndDate
No                     5174
2019-11-01 00:00:00     485
2019-12-01 00:00:00     466
2020-01-01 00:00:00     460
2019-10-01 00:00:00     458
Name: count, dtype: int64


In [9]:
contract_df['EndDate'] = contract_df['EndDate'].replace('No', 1)
contract_df['EndDate'] = contract_df['EndDate'].where(contract_df['EndDate'] == 1, 0)
contract_df['EndDate'] = contract_df['EndDate'].astype(int)
print(contract_df['EndDate'].value_counts())

EndDate
1    5174
0    1869
Name: count, dtype: int64


In [10]:
contract_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customerID        7043 non-null   object        
 1   BeginDate         7043 non-null   datetime64[ns]
 2   EndDate           7043 non-null   int32         
 3   Type              7043 non-null   object        
 4   PaperlessBilling  7043 non-null   object        
 5   PaymentMethod     7043 non-null   object        
 6   MonthlyCharges    7043 non-null   float64       
 7   TotalCharges      7043 non-null   object        
dtypes: datetime64[ns](1), float64(1), int32(1), object(5)
memory usage: 412.8+ KB


#### Columna 'Type', 'PaperlessBilling' y 'PaymentMethod'
Se modificaron los valores de las columnas para respetar el formato __snakecase__. Finalmente se mostraron los valores únicos de cada columna. 

In [11]:
def snakecase(df, columnas):
    '''
    Función que cambia los valores tipo str a minúsculas a partir de columnas de un DataFrame
    '''
    for columna in columnas:
        df[columna] = df[columna].str.lower()
    return df

In [12]:
def unique(df, columnnas):
    '''
    Función que permite obtener los valores únicos de columnas específicas de un DataFrame
    '''
    unique_values = []
    for columna in columnnas:
        unique_values.append(df[columna].unique())
    return unique_values

In [13]:
#Creación de lista de columnas a cambiar a formato snakecase
contract_sc = ['Type', 'PaperlessBilling', 'PaymentMethod']

contract_df = snakecase(contract_df, contract_sc)

In [14]:
contract_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customerID        7043 non-null   object        
 1   BeginDate         7043 non-null   datetime64[ns]
 2   EndDate           7043 non-null   int32         
 3   Type              7043 non-null   object        
 4   PaperlessBilling  7043 non-null   object        
 5   PaymentMethod     7043 non-null   object        
 6   MonthlyCharges    7043 non-null   float64       
 7   TotalCharges      7043 non-null   object        
dtypes: datetime64[ns](1), float64(1), int32(1), object(5)
memory usage: 412.8+ KB


In [15]:
#Revisión de valores únicos por columna
display(unique(contract_df, contract_sc))

[array(['month-to-month', 'one year', 'two year'], dtype=object),
 array(['yes', 'no'], dtype=object),
 array(['electronic check', 'mailed check', 'bank transfer (automatic)',
        'credit card (automatic)'], dtype=object)]

Se eliminó el segmento de string '(automatico)' de la en el caso de las transferencias bancarias y las tarjetas de credito ya que no aportaban información útil pues no existian pagos similares que no fueran realizados de forma automática.

In [16]:
contract_df['PaymentMethod'] = contract_df['PaymentMethod'].replace('bank transfer (automatic)', 'bank transfer')
contract_df['PaymentMethod'] = contract_df['PaymentMethod'].replace('credit card (automatic)', 'credit card')
display(contract_df['PaymentMethod'].unique())

array(['electronic check', 'mailed check', 'bank transfer', 'credit card'],
      dtype=object)

In [17]:
display(contract_df)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,1,month-to-month,yes,electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,1,one year,no,mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,0,month-to-month,yes,mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,1,one year,no,bank transfer,42.30,1840.75
4,9237-HQITU,2019-09-01,0,month-to-month,yes,electronic check,70.70,151.65
...,...,...,...,...,...,...,...,...
7038,6840-RESVB,2018-02-01,1,one year,yes,mailed check,84.80,1990.5
7039,2234-XADUH,2014-02-01,1,one year,yes,credit card,103.20,7362.9
7040,4801-JZAZL,2019-03-01,1,month-to-month,yes,electronic check,29.60,346.45
7041,8361-LTMKD,2019-07-01,0,month-to-month,yes,mailed check,74.40,306.6


#### Columna 'TotalCharges'
Al tratar convertir el tipo de varaible de la columna 'TotalCharge' a __float__ usando el método __.to_numeric()__ se observó que se producia un error debido a la presencia e cadenas vacias __' '__ en la columna que no son señaladas por el método __.info()__ como valores ausentes. Por lo tanto se utilizó __.replace()__ para sustituir las cadenas vacias por __'0'__ y posteriormente utiizar __.to_numeric()__.

In [18]:
contract_df['TotalCharges'] = contract_df['TotalCharges'].replace(' ', '0')
contract_df['TotalCharges'] = pd.to_numeric(contract_df['TotalCharges'], errors='coerce')
contract_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customerID        7043 non-null   object        
 1   BeginDate         7043 non-null   datetime64[ns]
 2   EndDate           7043 non-null   int32         
 3   Type              7043 non-null   object        
 4   PaperlessBilling  7043 non-null   object        
 5   PaymentMethod     7043 non-null   object        
 6   MonthlyCharges    7043 non-null   float64       
 7   TotalCharges      7043 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int32(1), object(4)
memory usage: 412.8+ KB


#### Duplicados explícitos
Se utilizó el método __.duplicated__ para buscar duplicados explícitos en el dataframe.

In [19]:
contract_duplicates = contract_df.duplicated().sum()
print('Los valores duplicados en el DataFrame users son:', contract_duplicates)

Los valores duplicados en el DataFrame users son: 0


### DataFrame internet_df

In [20]:
internet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB


In [21]:
display(internet_df.head())

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No


El DataFrame __internet_df__ únicamente contenia valores tipo object (string) y no presentaba valores ausentes. Debido a que los valores tipo object tienen la primer letra en mayúscula puede ser que hubiera datos que tuvieran las respuestas con un formato diferente pudiendo ser todo en mayúsculas, todo en minúsuculas que hubiera cadenas vacias. Para explorar dicha posibilidad se creo la función __unique__ y se aplicó a cada columna del DataFrame excluyendo la columna __customerID__.

In [22]:
#Se eliminó la columna 'customerID'
columnas = internet_df.columns.drop('customerID')
print(columnas)

Index(['InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies'],
      dtype='object')


In [23]:
print(unique(internet_df, columnas))

[array(['DSL', 'Fiber optic'], dtype=object), array(['No', 'Yes'], dtype=object), array(['Yes', 'No'], dtype=object), array(['No', 'Yes'], dtype=object), array(['No', 'Yes'], dtype=object), array(['No', 'Yes'], dtype=object), array(['No', 'Yes'], dtype=object)]


### Cambio a formato snakecase

### Duplicados explícitos

### DataFrame personal_df

In [24]:
personal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB


In [25]:
display(personal_df.head())

,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


In [26]:
phone_df.describe()

,customerID,MultipleLines
count,6361,6361
unique,6361,2
top,5575-GNVDE,No
freq,1,3390


In [27]:
phone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB


In [28]:
display(phone_df.head())

,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes
